<a href="https://colab.research.google.com/github/OlimpiaGarcia/Book_algorithm/blob/main/Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports generales
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno  as msno
import ast
import xgboost as xgb

# imports de modelo
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler


# imports de metricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# imports de red neuronal
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

# conexión con google
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
ruta_dir_test = "/content/drive/MyDrive/Colab Notebooks/octavo/benji/reto /data/train"

In [3]:
data = pd.read_csv(os.path.join(ruta_dir_test, 'train_scaled.csv'), encoding='utf-8')

In [4]:
text = pd.read_csv(os.path.join(ruta_dir_test, 'train_text.csv'), encoding='utf-8')

# Hacer que ambos DF usen la columna ID como indice

In [5]:
data.set_index('id', inplace=True)
text.set_index('id', inplace=True)

# Implementar KNN

In [6]:
def recomendar_librosKNN(id_libro, df_numerico, df_text, n_recomendaciones=10):
  #  KNN
  knn = NearestNeighbors(
      metric='cosine',
      algorithm='auto')
  knn.fit(df_numerico)

  # Vector del libro base
  vector_libro = df_numerico.loc[[id_libro]]

  # Obtener vecinos (sumamos 1 porque el primer resultado será el libro mismo)
  distancias, indices = knn.kneighbors(vector_libro, n_neighbors=n_recomendaciones + 1)

  # Ignorar el primer resultado (es el propio libro)
  indices_recomendados = indices[0][1:]
  ids_recomendados = df_numerico.iloc[indices_recomendados].index

  # Combinar con texto y devolver
  recomendaciones = df_text.loc[ids_recomendados][['book_title', 'book_details']]
  return recomendaciones, ids_recomendados

## Correr KNN

In [7]:
# Solicitar cuantos libros parecidos quieres:
#recomendaciones = int(input("¿Cuántos libros parecidos quieres?"))

# el libro es harry Potter 5
tabla, id_recomenados = recomendar_librosKNN(10611, data, text, n_recomendaciones=10)
tabla

,book_title,book_details
id,,
12399,Harry Potter and the Goblet of Fire,It is the summer holidays and soon Harry Potte...
12693,Harry Potter and the Deathly Hallows,"Harry has been burdened with a dark, dangerous..."
7595,Harry Potter and the Sorcerer's Stone,Harry Potter has no idea how famous he is. Tha...
9483,The Tales of Beedle the Bard,"""The Tales of Beedle the Bard"" contains five r..."
10061,Harry Potter and the Cursed Child. Parts One a...,Based on an original new story by J.K. Rowling...
7671,Harry Potter and the Cursed Child: Parts One a...,"The eighth story, nineteen years later...It wa..."
8666,The Harry Potter Collection 1-4,"The exciting tales of Harry Potter, the young ..."
11682,"Harry Potter Boxed Set, Books 1-5",Box Set containing Harry Potter and the Sorcer...
7145,Harry Potter Schoolbooks: Quidditch Through th...,SHS0545007984 Features: -Subjects: Literacy Ce...


In [21]:
data.drop(columns='average_rating', inplace=True)

# cargamos el modelo

In [22]:
# cargar el modelo nuevamente de ser necesario:
modelo_cargado = load_model("mejor_modelo1.keras")
modelo_cargado2 = load_model("mejor_modelo2.keras")

# nueva data

In [25]:
new_data = pd.read_csv("/content/data_benji_test - Hoja 1.csv", index_col=False, header=None, encoding="utf-8")
new_text = pd.read_csv("/content/text_benji_test - Hoja 1.csv", index_col=False, header=None,encoding="utf-8" )

In [26]:
print(new_data.shape)
print(new_text.shape)

print(data.shape)
print(text.shape)

(9, 8351)
(9, 3)
(12980, 8351)
(12980, 3)


# Anadimos nueva data con la data vieja

In [27]:
new_data.columns = data.columns
df_actualizado = pd.concat([data, new_data], ignore_index=True)

In [28]:
new_text.columns = text.columns
df_actualizado_text = pd.concat([text, new_text], ignore_index=True)

In [29]:
df_actualizado

,num_pages,num_ratings,num_reviews,rating_5,rating_4,rating_3,rating_2,rating_1,12th Century,13th Century,...,محمد عبد الرحمن العريفي,محمد قطب,مصطفى صادق الرافعي,مصطفى محمود,نور عبدالمجيد,هبة سيد عبد العاطي,يحيى بن شرف النووي,يوسف زيدان,ปรีดี พนมยงค์,နုနုရည်အင်းဝ
0,-0.842623,-0.217577,-0.286589,-0.177570,-0.242676,-0.250705,-0.223988,-0.159908,-0.012414,-0.012414,...,-0.008778,-0.015205,-0.008778,-0.015205,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778
1,-0.399557,-0.249876,-0.317189,-0.211447,-0.269901,-0.284236,-0.250421,-0.178224,-0.012414,-0.012414,...,-0.008778,-0.015205,-0.008778,-0.015205,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778
2,-0.000006,-0.239183,-0.304165,-0.207334,-0.258941,-0.260049,-0.225720,-0.166120,-0.012414,-0.012414,...,-0.008778,-0.015205,-0.008778,-0.015205,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778
3,-0.411424,-0.268686,-0.323764,-0.229737,-0.290560,-0.301126,-0.260874,-0.184972,-0.012414,-0.012414,...,-0.008778,-0.015205,-0.008778,-0.015205,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778
4,0.162188,-0.227681,-0.261426,-0.208593,-0.236048,-0.231541,-0.211471,-0.165478,-0.012414,-0.012414,...,-0.008778,-0.015205,-0.008778,-0.015205,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12984,-0.320438,-0.266131,-0.325661,0.895113,-0.227264,-0.286931,-0.300526,-0.259343,-0.183794,-0.012414,...,-0.008778,-0.015204,-0.008778,-0.015204,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778
12985,-0.458896,-0.240506,-0.276031,-0.287741,-0.211930,-0.256107,-0.260194,-0.222990,-0.166228,-0.012414,...,-0.008778,-0.015204,-0.008778,-0.015204,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778
12986,0.280866,-0.268581,-0.323448,1.137750,-0.229451,-0.290612,-0.301146,-0.261207,-0.185079,-0.012414,...,-0.008778,-0.015204,-0.008778,-0.015204,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778
12987,-0.403512,3.071097,3.860633,0.045885,2.079362,4.020799,4.005070,2.808982,1.435286,-0.012414,...,-0.008778,-0.015204,-0.008778,-0.015204,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778


In [30]:
X = df_actualizado

In [31]:
X1 = df_actualizado.drop(columns= ['rating_5', 'rating_4', 'rating_3', 'rating_2', 'rating_1'])


# predecimos

In [38]:
# predecir modelo 1
y_pred_nn1 = modelo_cargado.predict(X)

406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [39]:
y_pred_nn1 = pd.DataFrame(y_pred_nn1, columns=['y_pred_nn1'])
y_pred_nn1

,y_pred_nn1
0,2.886607
1,1.213431
2,1.884948
3,2.982721
4,2.985212
...,...
12984,2.818208
12985,1.920477
12986,2.847813
12987,2.280542


In [40]:
# predecir modelo 2
y_pred2_nn2 = modelo_cargado2.predict(X1)

406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [41]:
y_pred2_nn2 = pd.DataFrame(y_pred2_nn2, columns=['y_pred_nn2'])
y_pred2_nn2

,y_pred_nn2
0,2.961157
1,2.024027
2,1.993223
3,3.006481
4,3.031995
...,...
12984,2.461905
12985,2.163287
12986,2.704268
12987,2.353469


# juntarlos

In [45]:
# Función para mapear el rating a su descripción
def mapear_rating(valor):
    if valor <= 2.5:
        return "Malo"
    elif valor <= 2.9:
        return "Regular"
    elif valor <= 4.0:
        return "Bueno"
    else:
        return "Excelente"

In [48]:
# Crear nuevo DataFrame base con títulos y detalles
resultado = text.loc[id_recomenados][['book_title', 'book_details']].copy()

# Agregar columnas de predicción de cada DataFrame

resultado['pred1'] = y_pred_nn1.loc[id_recomenados]
resultado['pred1_desc'] = resultado['pred1'].apply(mapear_rating)

resultado['pred2'] = y_pred2_nn2.loc[id_recomenados]
resultado['pred2_desc'] = resultado['pred2'].apply(mapear_rating)

resultado

,book_title,book_details,pred1,pred1_desc,pred2,pred2_desc
id,,,,,,
12399,Harry Potter and the Goblet of Fire,It is the summer holidays and soon Harry Potte...,3.142661,Bueno,2.810922,Regular
12693,Harry Potter and the Deathly Hallows,"Harry has been burdened with a dark, dangerous...",3.435789,Bueno,2.747830,Regular
7595,Harry Potter and the Sorcerer's Stone,Harry Potter has no idea how famous he is. Tha...,4.277556,Excelente,3.054358,Bueno
9483,The Tales of Beedle the Bard,"""The Tales of Beedle the Bard"" contains five r...",2.893830,Regular,2.914394,Bueno
10061,Harry Potter and the Cursed Child. Parts One a...,Based on an original new story by J.K. Rowling...,1.016538,Malo,0.917088,Malo
7671,Harry Potter and the Cursed Child: Parts One a...,"The eighth story, nineteen years later...It wa...",1.016083,Malo,0.916611,Malo
8666,The Harry Potter Collection 1-4,"The exciting tales of Harry Potter, the young ...",3.000010,Bueno,3.045781,Bueno
11682,"Harry Potter Boxed Set, Books 1-5",Box Set containing Harry Potter and the Sorcer...,2.905824,Bueno,2.981893,Bueno
7145,Harry Potter Schoolbooks: Quidditch Through th...,SHS0545007984 Features: -Subjects: Literacy Ce...,3.163144,Bueno,2.983563,Bueno
